In [1]:
import pandas as pd
import numpy as np
import rdkit.Chem as Chem
from stratified_continious_split import scsplit, ContinuousStratifiedKFold
from rdkit.Chem.Scaffolds.MurckoScaffold import MurckoScaffoldSmiles
from utils import standardize_df, add_features, set_seeds
from sklearn.model_selection import GroupShuffleSplit, GroupKFold
from ray import tune
from ray.tune.search.optuna import OptunaSearch
from feature_pipeline import get_pipeline, get_pipeline_param_space
from utils import calc_scores

set_seeds(42)

In [2]:
def get_scaffold(smi) -> str:
    """
    Generate the Bemis-Murcko scaffold for a given molecule.

    :param smi: A SMILES string or an RDKit molecule object representing the
                molecule for which to generate the scaffold.
    :return: A SMILES string representing the Bemis-Murcko scaffold of the input
             molecule. If the scaffold cannot be generated, the input SMILES
             string is returned.
    """
    scaffold = MurckoScaffoldSmiles(smi)
    if len(scaffold) == 0:
        scaffold = smi
    return scaffold

In [3]:
def dedupplicate_parasite(df, duplicate_selection_criteria):
    uniques = (
        df.groupby("inchi")
        .filter(lambda x: len(x) == 1)[["inchi", "par_inhibition_per"]]
        .reset_index(drop=True)
    )
    duplicates = df.groupby("inchi").filter(lambda x: len(x) > 1).reset_index(drop=True)
    deduped = (
        # The inchi keys are the index after groupby; do not drop them but reset the index
        duplicates.groupby("inchi").agg(duplicate_selection_criteria).reset_index()
    )

    return pd.concat([uniques, deduped]).reset_index(drop=True)

In [4]:
derbyshire_df = pd.read_csv('./datasets/Derbyshire_reg_chembl_scores_corrected.csv').reset_index(drop=True)
derbyshire_df = derbyshire_df[['Compound SMILES', 'parasite % average']]

derbyshire_df = derbyshire_df.rename({
    'Compound SMILES': 'smiles',
    'parasite % average': 'par_inhibition_per',
}, axis=1)

derbyshire_df = standardize_df(derbyshire_df)
derbyshire_df = derbyshire_df[~derbyshire_df['inchi'].isna()].reset_index(drop=True)

duplicate_selection_criteria = {'par_inhibition_per': np.min}
derbyshire_df = dedupplicate_parasite(derbyshire_df, duplicate_selection_criteria)
derbyshire_df = add_features(derbyshire_df).dropna(axis=0).reset_index(drop=True)

clusters, _ = pd.factorize(
    derbyshire_df['mol']
        .map(Chem.MolToSmiles)
        .map(get_scaffold)
)
clusters = pd.Series(clusters)

derbyshire_df["inhibit_parasite"] = (derbyshire_df["par_inhibition_per"] <= 15.0).astype(float)

In [5]:
splitter = GroupShuffleSplit(n_splits=1, random_state=42)

X = derbyshire_df.drop(['inhibit_parasite', 'par_inhibition_per'], axis=1)
y = derbyshire_df['inhibit_parasite']
groups = clusters

train_val_idxs, test_idxs = next(splitter.split(X, y, groups))

X_train_val = X.loc[train_val_idxs].reset_index(drop=True)
y_train_val = y.loc[train_val_idxs].reset_index(drop=True)
groups_train_val = groups.loc[train_val_idxs].reset_index(drop=True)

X_test = X.loc[test_idxs].reset_index(drop=True)
y_test = y.loc[test_idxs].reset_index(drop=True)
groups_test = groups.loc[test_idxs].reset_index(drop=True)

In [6]:
def objective(config, X_train_val, y_train_val, groups_train_val):
    pipeline = get_pipeline()
    pipeline = pipeline.set_params(**config)

    kfold = GroupKFold(n_splits=3, shuffle=True, random_state=42)
    tally = []
    for train_idxs, val_idxs in kfold.split(X_train_val, y_train_val, groups_train_val):
        X_train, y_train = X_train_val.loc[train_idxs], y_train_val.loc[train_idxs]
        X_val, y_val = X_train_val.loc[val_idxs], y_train_val.loc[val_idxs]

        pipeline = pipeline.fit(X_train, y_train)
        y_val_pred_prob = pipeline.predict_proba(X_val)[:, 1]
        y_val_pred = np.where(y_val_pred_prob > 0.5, 1.0, 0.0)

        scores = calc_scores(y_val_pred_prob, y_val_pred, y_val)
        tally.append(scores)

    scores_tally = pd.DataFrame.from_records(tally)
    median_scores = scores_tally.median()
    return median_scores.to_dict()

In [7]:
tuner = tune.Tuner(
    tune.with_resources(
        tune.with_parameters(
            objective, 
            X_train_val=X_train_val, 
            y_train_val=y_train_val,
            groups_train_val=groups_train_val
        ),
        {'cpu': 4}
    ),
    param_space=get_pipeline_param_space(),
    tune_config=tune.TuneConfig(
        search_alg=OptunaSearch(seed=42),
        num_samples=200,
        metric="average_precision",
        mode="max",
    )
)

In [8]:
results = tuner.fit()

(objective pid=4308) /home/rahul_e_dev/micromamba/envs/bio/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
(objective pid=4308)   warnings.warn(
(objective pid=4308) /home/rahul_e_dev/micromamba/envs/bio/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
(objective pid=4308)   warnings.warn(
(objective pid=4308) /home/rahul_e_dev/micromamba/envs/bio/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
(objective pid=4308)   warnings.warn(
(objective pid=4308) /home/rahul_e_dev/micromamba/envs/bio/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will b

In [9]:
from sklearn.model_selection import TunedThresholdClassifierCV

best_result = results.get_best_result()
pipeline = get_pipeline()
config = {k:v for k,v in best_result.config.items() if k in pipeline.get_params()}
pipeline.set_params(**config)

pipeline = pipeline.fit(X_train_val, y_train_val)

kfold = ContinuousStratifiedKFold(n_splits=3)
th_pipeline = TunedThresholdClassifierCV(pipeline, scoring='f1', cv=kfold, thresholds=500)
th_pipeline.fit(X_train_val, y_train_val)

y_test_pred_prob = th_pipeline.predict_proba(X_test)[:, 1]
y_test_pred = th_pipeline.predict(X_test)
test_scores = calc_scores(y_test_pred_prob, y_test_pred, y_test.to_numpy())
test_scores = {k: np.round(v, 3) for k, v in test_scores.items()}
test_scores

/home/rahul_e_dev/micromamba/envs/bio/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/rahul_e_dev/micromamba/envs/bio/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/rahul_e_dev/micromamba/envs/bio/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/rahul_e_dev/micromamba/envs/bio/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/rahul_e_dev/micromamba/envs/bio/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarni

{'accuracy': 0.868,
 'balanced_accuracy': 0.648,
 'f1': 0.341,
 'precision': 0.31,
 'recall': 0.379,
 'roc_auc': 0.741,
 'average_precision': 0.314,
 'specificity': 0.916,
 'sensitivity': 0.379,
 'test_delong_auc': 0.741,
 'lb': 0.671,
 'ub': 0.812}

In [10]:
best_result.metrics_dataframe

,accuracy,balanced_accuracy,f1,precision,recall,roc_auc,average_precision,specificity,sensitivity,test_delong_auc,...,config/lgbmclassifier__reg_lambda,config/lgbmclassifier__num_leaves,config/lgbmclassifier__subsample,config/lgbmclassifier__colsample_bytree,config/lgbmclassifier__min_child_samples,config/lgbmclassifier__n_jobs,config/lgbmclassifier__random_state,config/lgbmclassifier__scale_pos_weight,config/lgbmclassifier__n_estimators,config/lgbmclassifier__max_depth
0,0.918619,0.531587,0.125,0.416667,0.074627,0.727096,0.235004,0.990629,0.074627,0.727096,...,0.000499,131,0.156771,0.194216,14,4,42,68,469,28
